In [ ]:
import pandas as pd
import numpy as np

### 入力パラメータ #####################################
prefix = './data/01_single/' # ファイルパス(任意)
ref = "C1"                   # 部品リファレンス

fname = prefix + ref + ".csv"                     # 入力ファイル名
fname_out = prefix + "output/" + ref + "_out.csv" # 出力ファイル名

# パラメータ情報を入力
cmax = 50   # 最大値
cmin = 0.25 # 最小値
step = 51   # ステップ数
IF = 50e6   # 中間周波数
#######################################################

# ヘッダを加えて読み込む
df = pd.read_csv(fname, header = None, names = ["ID", "Val", "VTUNE12", "Freq", "S21"])

# 先頭行と末尾行を削除する
# (元ファイルに余分な行が含まれるため)
df = df.drop(index=0)
df = df.drop(index=df.shape[0])

# IDは不要のため抹消する
df = df.drop(columns="ID")

# カラム名を上書きする
df_all = pd.DataFrame(columns=["Val","VTUNE12","fpeak","gain","fimg","img"])

# VTUNE12調整値を選択
for j, VTUNE in enumerate([0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5]):

    # 周波数ピークを検出
    for i, Cval in enumerate(range(step)):
    
        # 各ステップに変換
        Cval = Cval * (cmax-cmin)/cmax + cmin

        # ループで使用するデータフレームを抽出
        df_tmp = df.loc[df["VTUNE12"]==VTUNE]    
        df_tmp = df_tmp[df_tmp["Val"]==Cval]

        # ピーク値を取る
        tmp = df_tmp[df_tmp["S21"] == df_tmp["S21"].max()]
        fpeak = tmp["Freq"].tolist()[0]
        gain = tmp["S21"].tolist()[0]

        # イメージ周波数
        fimg = fpeak - IF * 2

        # イメージの計算
        if(fimg>(IF*2)):
            img = gain - df_tmp[df_tmp["Freq"]==fimg]["S21"].tolist()[0]
        else:
            img = "NG"

        # 元データに挿入
        df_all.loc[i+step*j,"Val"] = Cval
        df_all.loc[i+step*j,"VTUNE12"] = VTUNE
        df_all.loc[i+step*j,"fpeak"] = fpeak
        df_all.loc[i+step*j,"gain"] = gain
        df_all.loc[i+step*j,"fimg"] = fimg
        df_all.loc[i+step*j,"img"] = img

# 最終結果の出力
df_all.to_csv(fname_out)